In [1]:
!wget -nc "https://download.microsoft.com/download/D/4/6/D46FF87A-F6B9-4252-AA8B-3604ED519838/MSRParaphraseCorpus.msi"
!mkdir MRPC
!cabextract MSRParaphraseCorpus.msi -d MRPC
!cat MRPC/_2DEC3DBE877E4DB192D17C0256E90F1D | tr -d $'\r' > MRPC/msr_paraphrase_train.txt
!cat MRPC/_D7B391F9EAFF4B1B8BCE8F21B20B1B61 | tr -d $'\r' > MRPC/msr_paraphrase_test.txt
!rm MRPC/_*
!rm MSRParaphraseCorpus.msi

## dev ids from glue

!wget -nc 'https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc' -P MRPC/

--2020-10-23 10:37:23--  https://download.microsoft.com/download/D/4/6/D46FF87A-F6B9-4252-AA8B-3604ED519838/MSRParaphraseCorpus.msi
Resolving download.microsoft.com (download.microsoft.com)... 184.26.148.129, 2a02:26f0:ec:28c::e59, 2a02:26f0:ec:280::e59
Connecting to download.microsoft.com (download.microsoft.com)|184.26.148.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1359872 (1.3M) [application/octet-stream]
Saving to: ‘MSRParaphraseCorpus.msi’

MSRParaphraseCorpus 100%[===================>]   1.30M  --.-KB/s    in 0.05s   

2020-10-23 10:37:23 (24.0 MB/s) - ‘MSRParaphraseCorpus.msi’ saved [1359872/1359872]

mkdir: cannot create directory ‘MRPC’: File exists
Extracting cabinet: MSRParaphraseCorpus.msi
  extracting MRPC/_2D65ED66D69C42A28B021C3E24C1D8C0
  extracting MRPC/_2DEC3DBE877E4DB192D17C0256E90F1D
  extracting MRPC/_63DE49D9E7214609BE7E38DD145D8081
  extracting MRPC/_B3CFEFE1C368459BA1D1B8A2FA07A16D
  extracting MRPC/_C5BC91AAB1554DF3AF5E4105DE5

In [2]:
import pandas as pd


def get_as_csv(path_to_data):
    all_data = []

    with open(path_to_data, "r", encoding = 'utf-8-sig') as f:

        for _id, line in enumerate(f):

            if _id == 0:

                headers = line.split("\t")
            
            else:
                
                line = line.split("\t")

                all_data.append({
                    headers[0] : line[0],
                    headers[1] : line[1],
                    headers[2] : line[2],
                    headers[3] : line[3],
                    headers[4] : line[4]    
                })
            
    return pd.DataFrame(all_data)

In [3]:
train = get_as_csv("MRPC/msr_paraphrase_train.txt")
test = get_as_csv("MRPC/msr_paraphrase_test.txt")

In [4]:
def retrieve_dev(path_to_ids, train_pd):
    
    train_pd["id_pairs"] = train_pd.apply(lambda x: x["#1 ID"] + "_" + x["#2 ID"], axis = 1)
    
    ids_list = []
    
    with open(path_to_ids, "r", encoding = 'utf-8-sig') as f:
        
        for _id, line in enumerate(f):
            
            line = line.split("\t")
            
            id_1 = line[0]
            id_2 = line[1].strip()
            
            ids_list.append(id_1 + "_" + id_2)
            
    dev_pd = train_pd[train_pd.id_pairs.isin(ids_list)]
    train_new = train_pd[~train_pd.id_pairs.isin(ids_list)]
    
    assert len(train_new) + len(dev_pd) == len(train_pd)
    
    return train_new, dev_pd
    
train, dev = retrieve_dev("MRPC/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc", train)

In [5]:
def process_for_pipeline(df, exp_split):
    
    label_pairs = {"0":"semantic equivalence", "1":"paraphrase"}
    
    if "id_pairs" not in df.columns:
        
        df["id_pairs"] = df.apply(lambda x: x["#1 ID"] + "_" + x["#2 ID"], axis = 1)
    
    df = df.rename(columns = {
        "#1 ID" : "doc_id",
        "id_pairs" : "annotation_id",
        "#1 String" : "document",
        "#2 String\n" : "query",
        "Quality" : "label"
    })
    
    df = df.drop(columns = "#2 ID")
    df["label_id"] = df.label.apply(lambda x : label_pairs[x])
    df["exp_split"] = exp_split

    return df

train = process_for_pipeline(train, "train")
dev = process_for_pipeline(dev, "dev")
test = process_for_pipeline(test, "test")

In [6]:
import os
os.makedirs('data', exist_ok=True)
train.to_csv("data/train.csv", index = False)
dev.to_csv("data/dev.csv", index = False)
test.to_csv("data/test.csv", index = False)